#Set-Up

---
## Optimization Model 1 - Allocation Model for Store Space Optimization &

---


In [1]:
#Copy-and-paste the code below to use as "set-up" when your optimization model uses Pyomo and Coin-OR solvers.
#for reference, see https://jckantor.github.io/ND-Pyomo-Cookbook/notebooks/01.02-Running-Pyomo-on-Google-Colab.html#installing-pyomo-and-solvers

%%capture
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

from pyomo.environ import *

In [2]:
import pandas as pd
import csv

In [5]:
# Load data from CSV file
items_data = pd.read_csv("store_data.csv")

# Extract unique locations
unique_locations = set(items_data['location'].unique())

# Create a model for each location
models = {}
for location in unique_locations:
    model = ConcreteModel()

    # Filter items for the specific location
    location_items = items_data[items_data['location'] == location].to_dict('index')
    # Extract unique categories for this location
    unique_categories = set(items_data[items_data['location'] == location]['category'].unique())

    # Sets
    model.ITEMS = Set(initialize=location_items.keys())

    # Parameters
    model.depth = Param(model.ITEMS, initialize=lambda model, i: location_items[i]['depth'])
    model.width = Param(model.ITEMS, initialize=lambda model, i: location_items[i]['width'])
    model.profit = Param(model.ITEMS, initialize=lambda model, i: location_items[i]['profit'])

    # Variables
    model.space = Var(model.ITEMS, within=Binary)

    # Objective function: maximize total profit
    model.obj = Objective(expr=sum(model.profit[i] * model.space[i] for i in model.ITEMS), sense=maximize)

    # Constraint: total space available in the store for each location
    total_available_space = 1000000
    allocation_factor = 0.8  # Allocate 80% of the total required space
    model.space_constraint = Constraint(expr=sum(model.width[i] * model.depth[i] * model.space[i] for i in model.ITEMS) <= total_available_space * allocation_factor)

    # Constraint: total profit constraint
    total_desired_profit = 20000  # total minimum desired profit per location
    model.profit_constraint = Constraint(expr=sum(model.profit[i] * model.space[i] for i in model.ITEMS) >= total_desired_profit)

    # Sets, Parameters, Variables, Objective, and Constraints for this location
    for category in unique_categories:
        # Filter items for the specific category in this location
        items_in_category = [i for i in model.ITEMS if location_items[i]['category'] == category]

        # Ensure at most 10 items are selected from each category
        model.add_component(f"category_constraint_{category}", Constraint(expr=sum(model.space[i] for i in items_in_category) <= 10))

    # Solver
    solver = SolverFactory('cbc')
    solver.solve(model)

    # Store the model for this location
    models[location] = model

# Print profit for each location before storing in CSV
for location, model in models.items():
    total_profit = sum(model.profit[i] * model.space[i].value if model.space[i].value is not None else 0 for i in model.ITEMS)
    print(f"Total profit for {location}: {total_profit}")

# Print allocated items and profit for each location before storing in CSV
for location, model in models.items():
    print(f"Results for {location}:")
    total_profit = 0
    for i in model.ITEMS:
        item_name = location_items[i]['name'] if i in location_items else f'Item {i}'
        allocation_status = model.space[i].value if model.space[i].value is not None else None
        allocation_value = allocation_status if allocation_status is not None else 0
        total_profit += model.profit[i] * allocation_value
        print(f"Item {i}: {item_name} - Allocated: {allocation_value}")
    print(f"Total profit for {location}: {total_profit}")

# Store results in a DataFrame
results_data = []

for location, model in models.items():
    for i in model.ITEMS:
        item_name = location_items[i]['name'] if i in location_items else f'Item {i}'
        allocation_status = model.space[i].value if model.space[i].value is not None else None
        allocation_value = allocation_status if allocation_status is not None else 0

        # Extract the category for the item from the original dataset
        item_category = items_data.loc[i, 'category'] if i in items_data.index else None

        # Append the results to the list
        results_data.append({
            'Location': location,
            'Item': item_name,
            'Category': item_category,
            'Allocated': allocation_value,
            'Profit': model.profit[i] * allocation_value
        })

# Create a DataFrame from the results list
results_df = pd.DataFrame(results_data)

# Save the results to a CSV file
results_df.to_csv('allocation_results.csv', index=False)


Total profit for GA: 181301.5
Total profit for TX: 169216.8
Total profit for IL: 183055.5
Total profit for CA: 178356.0
Total profit for MA: 182970.5
Total profit for MN: 180241.5
Total profit for WI: 189244.0
Total profit for AZ: 170248.0
Results for GA:
Item 7: Item 7 - Allocated: 1.0
Item 15: Item 15 - Allocated: 1.0
Item 23: Item 23 - Allocated: 0.0
Item 31: Item 31 - Allocated: 0.0
Item 39: Item 39 - Allocated: 0.0
Item 47: Item 47 - Allocated: 1.0
Item 55: Item 55 - Allocated: 1.0
Item 63: Item 63 - Allocated: 1.0
Item 71: Item 71 - Allocated: 1.0
Item 79: Item 79 - Allocated: 0.0
Item 87: Item 87 - Allocated: 1.0
Item 95: Item 95 - Allocated: 1.0
Item 103: Item 103 - Allocated: 0.0
Item 111: Item 111 - Allocated: 0.0
Item 119: Item 119 - Allocated: 0.0
Item 127: Item 127 - Allocated: 0.0
Item 135: Item 135 - Allocated: 1.0
Item 143: Item 143 - Allocated: 0.0
Item 151: Item 151 - Allocated: 0.0
Item 159: Item 159 - Allocated: 0.0
Item 167: Item 167 - Allocated: 0.0
Item 175: Item

---
## Optimization Model 2 - Covering Model for Minimizing Transport & Storage Cost

---


In [6]:

# Cost matrix
cost_matrix = [
    [5, 7, 5, 8, 6],
    [10, 8, 8, 11, 5],
    [6, 6, 5, 6, 6],
    [10, 9, 8, 11, 7],
    [11, 5, 12, 12, 10],
    [11, 5, 7, 10, 7],
    [6, 8, 8, 12, 9],
    [8, 5, 10, 6, 10]
]

# Available inventory and warehouse capacity
available_inventory = [139, 143, 150, 147, 137, 144, 140, 137]
warehouse_capacity = [165, 198, 250, 174, 350]

# Check if the number of locations and warehouses match the cost matrix dimensions
assert len(available_inventory) == len(cost_matrix)
assert len(warehouse_capacity) == len(cost_matrix[0])

# Create a Pyomo Concrete Model
model = ConcreteModel()

# Sets
model.locations = range(len(available_inventory))
model.warehouses = range(len(warehouse_capacity))

# Decision variables
model.x = Var(model.locations, model.warehouses, within=NonNegativeIntegers)

# Objective function
model.obj = Objective(expr=sum(cost_matrix[i][j] * model.x[i, j] for i in model.locations for j in model.warehouses), sense=minimize)

# Constraints
model.inventory_constraint = ConstraintList()
for i in model.locations:
    model.inventory_constraint.add(expr=sum(model.x[i, j] for j in model.warehouses) == available_inventory[i])

model.capacity_constraint = ConstraintList()
for j in model.warehouses:
    model.capacity_constraint.add(expr=sum(model.x[i, j] for i in model.locations) <= warehouse_capacity[j])

# Solve the optimization problem using CBC solver
solver = SolverFactory('cbc')
solver.solve(model)

# Display the results
print("Optimal transport quantities:")
for i in model.locations:
    for j in model.warehouses:
        print(f"Location {i+1} to Warehouse {j+1}: {model.x[i, j].value}")

print("\nMinimum transport cost:", model.obj())

Optimal transport quantities:
Location 1 to Warehouse 1: 25.0
Location 1 to Warehouse 2: 0.0
Location 1 to Warehouse 3: 114.0
Location 1 to Warehouse 4: 0.0
Location 1 to Warehouse 5: 0.0
Location 2 to Warehouse 1: 0.0
Location 2 to Warehouse 2: 0.0
Location 2 to Warehouse 3: 0.0
Location 2 to Warehouse 4: 0.0
Location 2 to Warehouse 5: 143.0
Location 3 to Warehouse 1: 0.0
Location 3 to Warehouse 2: 0.0
Location 3 to Warehouse 3: 113.0
Location 3 to Warehouse 4: 37.0
Location 3 to Warehouse 5: 0.0
Location 4 to Warehouse 1: 0.0
Location 4 to Warehouse 2: 0.0
Location 4 to Warehouse 3: 0.0
Location 4 to Warehouse 4: 0.0
Location 4 to Warehouse 5: 147.0
Location 5 to Warehouse 1: 0.0
Location 5 to Warehouse 2: 137.0
Location 5 to Warehouse 3: 0.0
Location 5 to Warehouse 4: 0.0
Location 5 to Warehouse 5: 0.0
Location 6 to Warehouse 1: 0.0
Location 6 to Warehouse 2: 61.0
Location 6 to Warehouse 3: 23.0
Location 6 to Warehouse 4: 0.0
Location 6 to Warehouse 5: 60.0
Location 7 to Warehouse 1